单层神经网络 MNIST

In [1]:
#packages
import tensorflow as tf
import pandas as pd
from numpy import random as nr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder  
import types

In [18]:
#一些常量的定义
img_size=28*28

train_size=0.8
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
training_epochs=15
display_step=1
learning_rate=1e-2
batch_size=100

In [3]:
def read_data(filename):#用于读取数据的函数
    labeled_images = pd.read_csv(filename)
    [num_sample,num_feature]=labeled_images.shape
    images = labeled_images.iloc[:,1:]
    labels = labeled_images.iloc[:,:1]
    return (images,labels,num_sample,num_feature)

In [4]:
filename='./input/train.csv'
(images,labels,num_sample,num_feature)=read_data(filename)

In [5]:
#切割数据集
(train_images, test_images,train_labels, test_labels) = \
train_test_split(images, labels, train_size=train_size, random_state=0)
num_train=int(num_sample*train_size)

In [6]:
#将DataFrame转化为Matrix
training_images=train_images.as_matrix()
training_labels=train_labels.as_matrix()
testing_images=test_images.as_matrix()
testing_labels=test_labels.as_matrix()

In [7]:
training_images=training_images.astype('float32')#类型转换，以防matmul函数参数类型不匹配

In [8]:
#OneHot encoding
enc = OneHotEncoder()  
enc.fit(training_labels)  
print("enc.n_values_ is:",enc.n_values_  )
num_class=enc.n_values_[0]#类别数量
training_labels_onehot=enc.transform(training_labels).toarray()
testing_labels_onehot=enc.transform(testing_labels).toarray()

enc.n_values_ is: [10]


In [9]:
#create model
def multiNN(x,W,b):
    #Hidden layer1 with RELU activation
    layer_1=tf.matmul(x,W['w1'])+b['b1']
    layer_1=tf.nn.relu(layer_1)
    #Hidden layer1 with RELU activation
    layer_2=tf.matmul(layer_1,W['w2'])+b['b2']
    layer_2=tf.nn.relu(layer_2)
    #Output layer with linear activation
    out_layer=tf.matmul(layer_2,W['out'])+b['out']
    return out_layer

In [10]:
#Weights and bias
W={
    'w1':tf.Variable(tf.random_normal([img_size,n_hidden_1])),
    'w2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    'out':tf.Variable(tf.random_normal([n_hidden_2,num_class]))
}
b={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    'out':tf.Variable(tf.random_normal([num_class]))
}
#Construct model 
#[none,img_size]表示在tf运行时能输入任意形数量的images,每张image尺寸为img_size
x=tf.placeholder("float",[None,img_size])
y_ = tf.placeholder("float", [None,num_class])#正确的标签
y=multiNN(x,W,b)

In [19]:
#Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
#Initializing the variables
init=tf.global_variables_initializer()

In [20]:
#Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    #training cycle
    for epoch in range(training_epochs):
        avg_cost=0
        num_batch=int(num_train/batch_size)
        #loop over all batches
        for i in range(num_batch):
            batch_indices=nr.choice(range(num_train),batch_size)
            batch_xs=training_images[batch_indices,:]
            batch_ys=training_labels_onehot[batch_indices,:]
            _,c,W1,b1,y1,y2=sess.run([optimizer,cost,W,b,y,y_], feed_dict={x: batch_xs, y_: batch_ys})
            #compute average cost
            avg_cost+=c/num_batch
        #display logs
        if epoch % display_step == 0:
            print("Epoch:",'%04d' % (epoch+1),"cost=", \
                 "{:.9f}".format(avg_cost))
    print("Optimization Finished")
    #Test model
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print(accuracy.eval({x: testing_images, y_: testing_labels_onehot}))
            
    

Epoch: 0001 cost= 17208.922944205
Epoch: 0002 cost= 2976.359524977
Epoch: 0003 cost= 1703.998707419
Epoch: 0004 cost= 1010.151434243
Epoch: 0005 cost= 834.166687110
Epoch: 0006 cost= 680.451472038
Epoch: 0007 cost= 567.022684887
Epoch: 0008 cost= 667.106116865
Epoch: 0009 cost= 527.730698452
Epoch: 0010 cost= 537.871046027
Epoch: 0011 cost= 407.384948018
Epoch: 0012 cost= 476.508325008
Epoch: 0013 cost= 466.404713097
Epoch: 0014 cost= 379.277313279
Epoch: 0015 cost= 370.174721512
Optimization Finished
0.952262
